<a href="https://colab.research.google.com/github/hyxxnii/NLP-Practice/blob/master/Word%20Embedding/04.%20ELMo%20%ED%91%9C%ED%98%84%EC%9D%84%20%EC%82%AC%EC%9A%A9%ED%95%9C%20%EC%8A%A4%ED%8C%B8%20%EB%A9%94%EC%9D%BC%20%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference: https://wikidocs.net/33930

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


텐서플로우 허브로부터 다양한 사전 훈련된 모델(Pre-tained Model)들을 사용할 수 있다.

In [2]:
pip install tensorflow-hub

In [3]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K
import urllib.request
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [4]:
# 텐서플로우 허브로부터 ELMo 다운로드

elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)

sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

데이터셋 출처:  https://www.kaggle.com/uciml/sms-spam-collection-dataset

In [6]:
# 캐글의 스팸 메일 분류하기 데이터

urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [7]:
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
y_data = list(data['v1'])
X_data = list(data['v2'])

In [8]:
X_data[:5]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]

In [9]:
print(y_data[:5])

[0, 0, 1, 0, 0]


In [10]:
# 데이터 개수 확인

print(len(X_data))
n_of_train = int(len(X_data) * 0.8) # 80%
n_of_test = int(len(X_data) - n_of_train) # 20%
print(n_of_train)
print(n_of_test)

5572
4457
1115


In [11]:
X_train = np.asarray(X_data[:n_of_train]) #X_data 데이터 중에서 앞의 4457개의 데이터만 저장
y_train = np.asarray(y_data[:n_of_train]) #y_data 데이터 중에서 앞의 4457개의 데이터만 저장
X_test = np.asarray(X_data[n_of_train:]) #X_data 데이터 중에서 뒤의 1115개의 데이터만 저장
y_test = np.asarray(y_data[n_of_train:]) #y_data 데이터 중에서 뒤의 1115개의 데이터만 저장

ELMo와 설계한 모델을 연결하는 작업들을 진행해보자.

ELMo는 텐서플로우 허브로부터 가져온 것이기 때문에 케라스에서 사용하기 위해서는 케라스에서 사용할 수 있도록 변환해주는 작업들이 필요하다.

In [12]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict=True, signature="default")["default"]
    # 데이터의 이동이 케라스 → 텐서플로우 → 케라스가 되도록 하는 함수 

스팸 분류 모델은 ELMo를 이용한 임베딩 층을 거쳐서 256개의 뉴런이 있는 은닉층을 거친 후 마지막 1개의 뉴런을 통해 이진 분류를 수행한다.

In [13]:
from keras.models import Model
from keras.layers import Dense, Lambda, Input

input_text = Input(shape=(1,), dtype=tf.string)
# 케라스에서 중간에 텐서플로우의 레이어를 사용하고자 한다면 텐서플로우 레이어를 케라스의 Lambda 레이어로 감싸주어야 한다.
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(hidden_layer)
model = Model(inputs=[input_text], outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [14]:
history = model.fit(X_train, y_train, epochs=1, batch_size=60)

Epoch 1/1
4457/4457 [==============================] - 859s 193ms/step - loss: 0.1567 - accuracy: 0.9403


In [15]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

1115/1115 [==============================] - 202s 181ms/step

 테스트 정확도: 0.9731


1번의 epoch로 테스트 데이터 정확도 97%를 얻어내는 것을 확인할 수 있음